In [ ]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc, op, script, taproot

# Spending plan
* We have 20,000 sats in this output: 871864d7631024465fc210e553fa9f50e7f0f2359288ad121aa733d65e366995:0
* We want to spend all of it to tb1ptaqplrhnyh3kq85n7dtm5vcpgstt0ev80f4wd8ngeppch4fzu8mquchufq
* 1 input/1 output transaction


In [ ]:
# Spending from a p2tr
from ecc import PrivateKey, N
from hash import sha256
from helper import big_endian_to_int
from script import address_to_script_pubkey
from tx import Tx, TxIn, TxOut
my_email = b"jimmy@programmingblockchain.com"
my_secret = big_endian_to_int(sha256(my_email))
priv = PrivateKey(my_secret)
prev_tx = bytes.fromhex("871864d7631024465fc210e553fa9f50e7f0f2359288ad121aa733d65e366995")
prev_index = 0
target_address = "tb1ptaqplrhnyh3kq85n7dtm5vcpgstt0ev80f4wd8ngeppch4fzu8mquchufq"
fee = 500
tx_in = TxIn(prev_tx, prev_index)
target_script_pubkey = address_to_script_pubkey(target_address)
target_amount = tx_in.value(network="signet") - fee
tx_out = TxOut(target_amount, target_script_pubkey)
tx_obj = Tx(1, [tx_in], [tx_out], network="signet", segwit=True)
tweaked_secret = (priv.secret + big_endian_to_int(priv.point.tweak())) % N
tweaked_key = PrivateKey(tweaked_secret)
tx_obj.sign_p2tr_keypath(0, tweaked_key)
print(tx_obj.serialize().hex())

### Exercise 1




#### Make [this test](/edit/session2/ecc.py) pass: `ecc.py:PrivateKeyTest:test_tweaked_key`

In [ ]:
# Exercise 1

reload(ecc)
run(ecc.PrivateKeyTest('test_tweaked_key'))

### Exercise 2

## Checkpoint Exercise

You have been sent 100,000 sats to your address on Signet. Send 40,000 sats back to <code>tb1q7kn55vf3mmd40gyj46r245lw87dc6us5n50lrg</code>, the rest to yourself.

Use <a href="https://mempool.space/signet/tx/push" target="_mempool">Mempool Signet</a> to broadcast your transaction



In [ ]:
# Exercise 2

from ecc import PrivateKey
from hash import sha256
from helper import big_endian_to_int
from script import address_to_script_pubkey
from tx import Tx, TxIn, TxOut
my_email = b"<fill this in with your email>"
my_secret = big_endian_to_int(sha256(my_email))
priv = PrivateKey(my_secret)
target_address = "tb1q7kn55vf3mmd40gyj46r245lw87dc6us5n50lrg"
target_amount = 40000
fee = 500
# fill the next two variables from the block explorer
prev_tx = bytes.fromhex("<fill in from block explorer>")
prev_index = -1
# create the one input

# use the address_to_script_pubkey to get the ScriptPubKey

# create the target output

# calculate the change amount

# use the private key's point's p2tr_script method to get the change ScriptPubkey

# create the change output

# create the transaction

# sign the transaction using the tweaked key and the sign_p2tr_keypath method

# print the serialized hex


# OP_CHECKSIGADD
* Consumes the top three elements: a pubkey, a number, and a signature.
* Valid sig, returns the number+1 to the stack
* Invalid sig, returns the number back to the stack


In [ ]:
def op_checksigadd_schnorr(stack, tx_obj, input_index):
    # check to see if there's at least 3 elements
    if len(stack) < 3:
        return False
    # pop off the pubkey
    pubkey = stack.pop()
    # pop off the n and do decode_num on it
    n = decode_num(stack.pop())
    # pop off the signature
    sig = stack.pop()
    # parse the pubkey
    point = S256Point.parse_xonly(pubkey)
    # if the signature has 0 length, it is not valid
    # so put encode_num(n) back on stack and return True
    if len(sig) == 0:
        stack.append(encode_num(n))
        return True
    # use the get_signature_and_hashtype function on the sig
    schnorr, hash_type = get_signature_and_hashtype(sig)
    # get the message from the tx_obj.sig_hash using input index and hash type
    msg = tx_obj.sig_hash(input_index, hash_type)
    # verify the Schnorr signature
    if point.verify_schnorr(msg, schnorr):
        # if valid, increment the n, encode_num it and push back on stack
        stack.append(encode_num(n + 1))
    else:
        # if invalid, encode_num on n and push back on stack
        stack.append(encode_num(n))
    # return True for successful execution
    return True

# Example TapScripts
* 1-of-1 (pay-to-pubkey) [pubkey, OP_CHECKSIG]
* 2-of-2 [pubkey A, OP_CHECKSIGVERIFY, pubkey B, OP_CHECKSIG]
* 2-of-3 [pubkey A, OP_CHECKSIG, pubkey B, OP_CHECKSIGADD, pubkey C, OP_CHECKSIGADD, OP_2, OP_EQUAL]
* halvening timelock 1-of-1 [840000, OP_CHECKLOCKTIMEVERIFY, OP_DROP, pubkey, OP_CHECKSIG]


In [ ]:
# Example TapScripts
from ecc import PrivateKey
from op import encode_minimal_num
from taproot import TapScript
pubkey_a = PrivateKey(11111111).point.xonly()
pubkey_b = PrivateKey(22222222).point.xonly()
pubkey_c = PrivateKey(33333333).point.xonly()
# 1-of-1 (0xAC is OP_CHECKSIG)
tap_script = TapScript([pubkey_a, 0xAC])
print(tap_script)
# 2-of-2 (0xAD is OP_CHECKSIGVERIFY)
tap_script = TapScript([pubkey_a, 0xAD, pubkey_b, 0xAC])
print(tap_script)
# 2-of-3 (0xBA is OP_CHECKSIGADD, 0x52 is OP_2, 0x87 is OP_EQUAL)
tap_script = TapScript([pubkey_a, 0xAC, pubkey_b, 0xBA, pubkey_c, 0xBA, 0x52, 0x87])
print(tap_script)
# halvening timelock 1-of-1 (0xB1 is OP_CLTV, 0x75 is OP_DROP)
tap_script = TapScript([encode_minimal_num(840000), 0xB1, 0x75, pubkey_a, 0xAC])
print(tap_script)

### Exercise 3

Make a TapScript for 4-of-4 using pubkeys from private keys which correspond to 10101, 20202, 30303, 40404



In [ ]:
# Exercise 3

from ecc import PrivateKey
from taproot import TapScript
pubkey_1 = PrivateKey(10101).point.xonly()
pubkey_2 = PrivateKey(20202).point.xonly()
pubkey_3 = PrivateKey(30303).point.xonly()
pubkey_4 = PrivateKey(40404).point.xonly()
# create a 4-of-4 tapscript that uses OP_CHECKSIGVERIFY (0xad) and OP_CHECKSIG (0xac)

# print the TapScript


# TapLeaf
* These are the leaves of the Merkle Tree
* Has a TapLeaf Version (<code>0xc0</code>) and TapScript
* Any Leaf can be executed to satisfy the Taproot Script Path
* Hash of a TapLeaf is a Tagged Hash (TapLeaf) of the version + TapScript


In [ ]:
# Example of making a TapLeaf and calculating the hash
from ecc import PrivateKey
from hash import hash_tapleaf
from helper import int_to_byte
from taproot import TapScript, TapLeaf
pubkey_a = PrivateKey(11111111).point.xonly()
pubkey_b = PrivateKey(22222222).point.xonly()
tap_script = TapScript([pubkey_a, 0xAD, pubkey_b, 0xAC])
tap_leaf = TapLeaf(tap_script)
h = hash_tapleaf(int_to_byte(tap_leaf.tapleaf_version) + tap_leaf.tap_script.serialize())
print(h.hex())

### Exercise 4

Calculate the TapLeaf hash whose TapScript is a 2-of-4 using pubkeys from private keys which correspond to 10101, 20202, 30303, 40404



In [ ]:
# Exercise 4

from ecc import PrivateKey
from hash import hash_tapleaf
from helper import int_to_byte
from taproot import TapScript, TapLeaf
pubkey_1 = PrivateKey(10101).point.xonly()
pubkey_2 = PrivateKey(20202).point.xonly()
pubkey_3 = PrivateKey(30303).point.xonly()
pubkey_4 = PrivateKey(40404).point.xonly()
# create a 2-of-4 TapScript that uses OP_CHECKSIG (0xac), OP_CHECKSIGADD (0xba), OP_2 (0x52) and OP_EQUAL (0x87)

# create the TapLeaf with the TapScript

# calculate the hash by using hash_tapleaf on the tapleaf version and the tap script

# print the hash hex


### Exercise 5




#### Make [this test](/edit/session2/taproot.py) pass: `taproot.py:TapRootTest:test_tapleaf_hash`

In [ ]:
# Exercise 5

reload(taproot)
run(taproot.TapRootTest('test_tapleaf_hash'))

# TapBranch
* These are the branches of the Merkle Tree
* These connect a left child and a right child.
* Each child is a TapLeaf or TapBranch
* Hash of a TapBranch is a Tagged Hash (TapBranch) of the left hash and right hash, sorted


In [ ]:
# Example of making a TapBranch and calculating the hash
from ecc import PrivateKey
from hash import hash_tapbranch
from helper import int_to_byte
from taproot import TapScript, TapLeaf, TapBranch
pubkey_1 = PrivateKey(11111111).point.xonly()
pubkey_2 = PrivateKey(22222222).point.xonly()
tap_script_1 = TapScript([pubkey_1, 0xAC])
tap_script_2 = TapScript([pubkey_2, 0xAC])
tap_leaf_1 = TapLeaf(tap_script_1)
tap_leaf_2 = TapLeaf(tap_script_2)
tap_branch = TapBranch(tap_leaf_1, tap_leaf_2)
left_hash = tap_branch.left.hash()
right_hash = tap_branch.right.hash()
if left_hash > right_hash:
    h = hash_tapbranch(left_hash + right_hash)
else:
    h = hash_tapbranch(right_hash + left_hash)
print(h.hex())

### Exercise 6

TabBranch Calculation

Calculate the TabBranch hash whose left and right nodes are TapLeafs whose TapScripts are for a 1-of-2 using pubkeys from private keys which correspond to (10101, 20202) for the left, (30303, 40404) for the right



In [ ]:
# Exercise 6

from ecc import PrivateKey
from hash import hash_tapbranch
from helper import int_to_byte
from taproot import TapScript, TapLeaf, TapBranch
pubkey_1 = PrivateKey(10101).point.xonly()
pubkey_2 = PrivateKey(20202).point.xonly()
pubkey_3 = PrivateKey(30303).point.xonly()
pubkey_4 = PrivateKey(40404).point.xonly()
# create two 1-of-2 TapScripts


# create two TapLeafs with the TapScripts


# create the branch

# get the left and right hashes


# calculate the hash using the sorted order with hash_tapbranch




# print the hex of the hash


### Exercise 7




#### Make [this test](/edit/session2/taproot.py) pass: `taproot.py:TapRootTest:test_tapbranch_hash`

In [ ]:
# Exercise 7

reload(taproot)
run(taproot.TapRootTest('test_tapbranch_hash'))

# Computing the Merkle Root
* The Merkle Root is the hash of the root element of the Merkle Tree
* For TapLeaf: TapLeafHash(version + TapScript serialization)
* For TapBranch: TapBranchHash(sorted(left, right))
* It doesn't have to be the hash of anything, just has to be 32 bytes


In [ ]:
# Example of Comupting the Merkle Root
from ecc import PrivateKey
from hash import hash_tapbranch
from helper import int_to_byte
from taproot import TapScript, TapLeaf, TapBranch
pubkey_1 = PrivateKey(11111111).point.xonly()
pubkey_2 = PrivateKey(22222222).point.xonly()
pubkey_3 = PrivateKey(33333333).point.xonly()
tap_script_1 = TapScript([pubkey_1, 0xAC])
tap_script_2 = TapScript([pubkey_2, 0xAC])
tap_script_3 = TapScript([pubkey_3, 0xAC])
tap_leaf_1 = TapLeaf(tap_script_1)
tap_leaf_2 = TapLeaf(tap_script_2)
tap_leaf_3 = TapLeaf(tap_script_3)
tap_branch_1 = TapBranch(tap_leaf_1, tap_leaf_2)
tap_root = TapBranch(tap_branch_1, tap_leaf_3)
merkle_root = tap_root.hash()
print(merkle_root.hex())

### Exercise 8

Calculate the External PubKey for a Taproot output whose internal pubkey is 90909 and whose Merkle Root is from two TapBranches, each of which is a single signature TapLeaf. The private keys corresponding to the left TapBranch's TapLeafs are 10101 and 20202. The private keys corresponding to the right TapBranch's TapLeafs are 30303 and 40404.



In [ ]:
# Exercise 8

from ecc import PrivateKey
from helper import big_endian_to_int
from taproot import TapScript, TapLeaf, TapBranch
internal_pubkey = PrivateKey(90909).point
pubkey_1 = PrivateKey(10101).point.xonly()
pubkey_2 = PrivateKey(20202).point.xonly()
pubkey_3 = PrivateKey(30303).point.xonly()
pubkey_4 = PrivateKey(40404).point.xonly()
# create four tap scripts [pubkey, 0xac] one for each pubkey




# create four TapLeafs with the TapScripts




# create two TapBranches that have these TapLeafs


# create another TapBranch that corresponds to the merkle root and get its hash

# the external public key is the internal public key tweaked with the Merkle Root

# print the hex of the xonly of the external pubkey


# Control Block
* Required for spending a TapScript, last element of Witness
* TapScript Version (<code>0xc0</code> or <code>0xc1</code>)
* The last bit expresses the parity of the external pubkey, which is necessary for batch verification
* Internal PubKey $P$
* Merkle Proof (list of hashes to combine to get to the Merkle Root)


# Merkle Proof
* List of hashes
* Combine each with the hash of the TapScript, sorting them each time
* The result is the Merkle Root $m$, which is combined with the Internal PubKey $P$ to get the tweak $t$ with the formula $t=H_{taptweak}(P+m)$
* If the result of $P+tG=Q$ where $Q$ is the External PubKey from the UTXO, this is a valid TapScript


In [ ]:
# Example of Control Block Validation
from ecc import PrivateKey, S256Point
from hash import hash_tapbranch
from helper import int_to_byte
from taproot import TapScript, TapLeaf, TapBranch
external_pubkey_xonly = bytes.fromhex("cbe433288ae1eede1f24818f08046d4e647fef808cfbbffc7d10f24a698eecfd")
pubkey_2 = bytes.fromhex("027aa71d9cdb31cd8fe037a6f441e624fe478a2deece7affa840312b14e971a4")
tap_script_2 = TapScript([pubkey_2, 0xAC])
tap_leaf_2 = TapLeaf(tap_script_2)
tap_leaf_1_hash = bytes.fromhex("76f5c1cdfc8b07dc8edca5bef2b4991201c5a0e18b1dbbcfe00ef2295b8f6dff")
tap_leaf_3_hash = bytes.fromhex("5dd270ec91aa5644d907059400edfd98e307a6f1c6fe3a2d1d4550674ff6bc6e")
internal_pubkey = S256Point.parse(bytes.fromhex("407910a4cfa5fe195ad4844b6069489fcb429f27dff811c65e99f7d776e943e5"))
current = tap_leaf_2.hash()
for h in (tap_leaf_1_hash, tap_leaf_3_hash):
    if h < current:
        current = hash_tapbranch(h + current)
    else:
        current = hash_tapbranch(current + h)
print(internal_pubkey.tweaked_key(current).xonly() == external_pubkey_xonly)
print(internal_pubkey.p2tr_address(current, network="signet"))

### Exercise 9

Validate the Control Block for the pubkey whose private key is 40404 for the external pubkey from the last exercise



In [ ]:
# Exercise 9

from ecc import PrivateKey, S256Point
from helper import big_endian_to_int
from taproot import TapScript, TapLeaf, TapBranch
external_pubkey_xonly = bytes.fromhex("8b9f09cd4a33e62b0c9d086056bbdeb7a218c1e4830291b9be56841b31d94ccb")
internal_pubkey = PrivateKey(90909).point
hash_1 = bytes.fromhex("22cac0b60bc7344152a8736425efd62532ee4d83e3de473ed82a64383b4e1208")
hash_2 = bytes.fromhex("a41d343d7419b99bfe8e66752fc3c45fd14aa2cc5ef5bf9073ed28dfc60e2e34")
pubkey_4 = bytes.fromhex("9e5f5a5c29d33c32185a3dc0a9ccb3e72743744dd869dd40b6265a23fd84a402")
# create the TapScript and TapLeaf for pubkey 4 using [pubkey, 0xac]


# set the current hash to the TapLeaf's hash

# loop through hash_1 and hash_2

    # update current hash to be the hash_tapbranch of h and the current hash, sorted alphabetically




# get the external pubkey is the internal pubkey's tweaked_key method with the current hash

# check to see if the external pubkey's xonly is correct


### Exercise 10




#### Make [this test](/edit/session2/taproot.py) pass: `taproot.py:TapRootTest:test_control_block`

In [ ]:
# Exercise 10

reload(taproot)
run(taproot.TapRootTest('test_control_block'))

### Exercise 11

Create a Signet P2TR address with these Script Spend conditions:

1. Internal Public Key is <code>cd04c1bf88ca891af152fc57c36523ab59efb16b7ec07caca0cfc4a1f2051d9e</code>
2. Leaf 1 and Leaf 2 make Branch 1, Branch 1 and Leaf 3 make Branch 2, which is the Merkle Root
3. All TapLeaf are single key locked TapScripts (pubkey, OP_CHECKSIG)
4. Leaf 1 uses your xonly pubkey
5. Leaf 2 uses this xonly pubkey: <code>331a8f6a14e1b41a6b523ddb505fbc0662a6446bd42408692497297d3474aeec</code>
6. Leaf 3 uses this xonly pubkey: <code>158a49d62c384c539a453e41a70214cfb85184954ae5c8df4b47eb74d58ff16f</code>



In [ ]:
# Exercise 11

from ecc import PrivateKey, S256Point
from hash import sha256
from helper import big_endian_to_int
from taproot import TapScript, TapLeaf, TapBranch
my_secret = b"<fill this in with your email>"
my_secret = big_endian_to_int(sha256(my_email))
internal_pubkey = S256Point.parse(bytes.fromhex("cd04c1bf88ca891af152fc57c36523ab59efb16b7ec07caca0cfc4a1f2051d9e"))
pubkey_2 = bytes.fromhex("331a8f6a14e1b41a6b523ddb505fbc0662a6446bd42408692497297d3474aeec")
pubkey_3 = bytes.fromhex("158a49d62c384c539a453e41a70214cfb85184954ae5c8df4b47eb74d58ff16f")
# get your xonly pubkey using PrivateKey

# make the first TapScript and TapLeaf using your xonly and OP_CHECKSIG (0xAC)


# make the second and third TapLeaves using pubkey_2 and pubkey_3 respectively




# make a TapBranch with leaf 1 and 2

# make a TapBranch with branch 1 and leaf 3

# get the hash of this branch, this is the Merkle Root

# print the address using the p2tr_address method of internal_pubkey and specify signet


### Exercise 12

Send yourself the rest of the coins from the output of the previous exercise to the address you just created

Use <a href="https://mempool.space/signet/tx/push" target="_mempool">Mempool Signet</a> to broadcast your transaction



In [ ]:
# Exercise 12

from ecc import PrivateKey, S256Point
from hash import sha256
from helper import big_endian_to_int
from script import address_to_script_pubkey
from taproot import TapScript, TapLeaf, TapBranch
from tx import Tx, TxIn, TxOut
my_secret = b"<fill this in with your email>"
my_secret = big_endian_to_int(sha256(my_email))
my_private_key = PrivateKey(my_secret)
prev_tx = bytes.fromhex("<fill this in with the tx where you spent last time>")
prev_index = 1
target_address = "<fill this in with the address from the last exercise>"
fee = 500
# create a transaction input

# calculate the target amount


# create a transaction output

# create a transaction, segwit=True and network="signet"

# calculate the tweaked key from your private key

# sign the transaction using sign_p2tr_keypath

# print the serialized hex


### Exercise 13

Now spend this output using the script path from the second TapLeaf send it all to <code>tb1q7kn55vf3mmd40gyj46r245lw87dc6us5n50lrg</code>

Use <a href="https://mempool.space/signet/tx/push to broadcast your transaction" target="_mempool">Mempool Signet</a> to broadcast your transaction



In [ ]:
# Exercise 13

from ecc import PrivateKey, S256Point
from hash import sha256
from helper import big_endian_to_int
from script import address_to_script_pubkey
from taproot import TapScript, TapLeaf, TapBranch
from tx import Tx, TxIn, TxOut
from witness import Witness
my_email = b"<fill this in with your email>"
my_secret = big_endian_to_int(sha256(my_email))
my_private_key = PrivateKey(my_secret)
internal_pubkey = S256Point.parse(bytes.fromhex("cd04c1bf88ca891af152fc57c36523ab59efb16b7ec07caca0cfc4a1f2051d9e"))
pubkey_2 = bytes.fromhex("331a8f6a14e1b41a6b523ddb505fbc0662a6446bd42408692497297d3474aeec")
pubkey_3 = bytes.fromhex("158a49d62c384c539a453e41a70214cfb85184954ae5c8df4b47eb74d58ff16f")
my_xonly = my_private_key.point.xonly()
tap_script_1 = TapScript([my_xonly, 0xAC])
tap_leaf_1 = TapLeaf(tap_script_1)
tap_script_2 = TapScript([pubkey_2, 0xAC])
tap_leaf_2 = TapLeaf(tap_script_2)
tap_script_3 = TapScript([pubkey_3, 0xAC])
tap_leaf_3 = TapLeaf(tap_script_3)
prev_tx = bytes.fromhex("<fill this in with the tx you just submitted>")
prev_index = 0
target_address = "tb1q7kn55vf3mmd40gyj46r245lw87dc6us5n50lrg"
fee = 500
# create the two branches needed (leaf 1, leaf 2), (branch 1, leaf 3)


# create a transaction input

# calculate the target amount

# calculate the target script

# create a transaction output

# create a transaction, segwit=True and network="signet"

# create the control block from the TapBranch control_block method with internal_pubkey and tap_leaf_1
cb = tap_branch_2.control_block(internal_pubkey, tap_leaf_1)
# set the tx's witness to be the tap_script_1.raw_serialize() and control block, serialized
tx_in.witness = Witness([tap_script_1.raw_serialize(), cb.serialize()])
# set the message to be the transaction's sig_hash at index 0
msg = tx_obj.sig_hash(0)
# use sign_schnorr with yoru private key on the message and serialize it to get the signature
sig = my_private_key.sign_schnorr(msg).serialize()
# insert the sig in front of the other elements in the witness using tx_in.witness.items.insert
tx_in.witness.items.insert(0, sig)
# verify the transaction
print(tx_obj.verify())
# print the serialized hex
print(tx_obj.serialize().hex())